In [ ]:
import numpy as np
import sep

from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

with fits.open("../data/image.fits") as hdul:
    data = hdul[0].data

# Display the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.title("Input Image");

# Estimate and plot the background
bkg = sep.Background(data)
print(f"Global background: {bkg.globalback}")
print(f"Global RMS: {bkg.globalrms}")

# Background image
bkg_image = bkg.back()
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.title("Background Image");

bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.title("Background RMS");

# Subtract the background from the data
data_sub = data - bkg

# Object detection
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
print(f"Number of objects detected: {len(objects)}")

# Plot the detected objects
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# Overlay the ellipses for the detected objects
for obj in objects:
    e = Ellipse(xy=(obj['x'], obj['y']),
                width=6*obj['a'],
                height=6*obj['b'],
                angle=obj['theta'] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.title("Detected Objects");

# the aperture photometry
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

for i in range(min(10, len(flux))):
    print(f"Object {i}: flux = {flux[i]:.2f} ± {fluxerr[i]:.2f}")

print("Object properties:", objects.dtype.names)
